In [1]:
import numpy as np
import pandas as pd
import os
import re

from functools import reduce

In [2]:
DATA_PATH = "technology_1m_prices.csv"

In [3]:
data = pd.read_csv(DATA_PATH, index_col=0)
data.dropna(inplace=True)
data

FileNotFoundError: [Errno 2] No such file or directory: 'technology_1m_prices.csv'

In [4]:
data["ticker"].unique()

array(['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'FB', 'TSM'],
      dtype=object)

In [5]:
close_prices = []
for ticker in data["ticker"].unique():
    close_prices.append(data[data["ticker"] == ticker][["t", "c", "ticker"]].set_index("t"))

In [6]:
current_df = close_prices[0]
for i in range(1, len(close_prices)):
    current_df = current_df.join(close_prices[i], rsuffix=f"_{close_prices[i]['ticker'].iloc[0]}", how="inner")

In [7]:
current_df = current_df.rename(columns={"c": "c_AAPL"}).drop([col for col in current_df.columns if re.search("ticker.*", col)], axis=1)

In [8]:
current_df.corr()

,c_AAPL,c_MSFT,c_GOOG,c_AMZN,c_TSLA,c_NVDA,c_FB,c_TSM
c_AAPL,1.000000,0.897821,0.827041,0.316205,0.827706,0.888518,0.421611,0.593046
c_MSFT,0.897821,1.000000,0.954454,0.478184,0.812934,0.951964,0.643755,0.527825
c_GOOG,0.827041,0.954454,1.000000,0.495090,0.718925,0.856801,0.729178,0.604520
c_AMZN,0.316205,0.478184,0.495090,1.000000,0.283836,0.431221,0.647779,0.264214
c_TSLA,0.827706,0.812934,0.718925,0.283836,1.000000,0.804004,0.284511,0.702535
c_NVDA,0.888518,0.951964,0.856801,0.431221,0.804004,1.000000,0.525818,0.411075
c_FB,0.421611,0.643755,0.729178,0.647779,0.284511,0.525818,1.000000,0.290485
c_TSM,0.593046,0.527825,0.604520,0.264214,0.702535,0.411075,0.290485,1.000000


Как видно, акции `AAPL` сильно скоррелированы с акциями `MSFT`, `GOOG`, `TSLA`, `NVDA`.